# Counting corners in polygons

In this optional exercise you will: 

* Train a CNN to count corners in polygons

## The data

The images are almost black and white images of polygons, from triangles up to polygons with 10 corners.
The task is simply to be able to predict the number of corners given an input image. The size of the 
images are 100x100. There is a training dataset of 5000 images and a testset of also 5000 images. There is also
a dataset where training data only contain odd number of corners and testdata only even number of corners.

## The exercises
See cell # below.


### CellName: Init (#1)
### CellType: Needed
### Cell instruction: Initializing the libraries
In the cell below, we import all the libraries that are needed for this exercises. There is one configuration parameter that you can change in this cell

* Inline or "pop out" plots.

See comments in the cell for more information. Run the cell by entering into the cell and press "CTRL Enter".


In [ ]:
import numpy as np
import tensorflow as tf
import time
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""

import keras
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import TimeDistributed
from keras.layers import Lambda, concatenate
from keras.layers import LSTM, GRU, SimpleRNN, RNN

from keras.optimizers import SGD, Adam, RMSprop, Nadam
from keras import backend as K

from sklearn.metrics import *

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# To have the plots inside the notebook "inlin" should be True. 
# If "inlin" = False, then plots will pop out of the notebook
inlin = True # True/False
if inlin:
    %matplotlib inline
else:
    %matplotlib

import matplotlib
import matplotlib.pyplot as plt


### CellName: Data (#2)
### CellType: Needed
### Cell instruction: Function for loading the polygons

Run the cell by entering into the cell and press "CTRL Enter".

In [ ]:
def loadImages(folder,trgFile,n):
    from scipy import misc
    def load_pics(folder,n):
        imgs = []
        for i in range(n):
            img = misc.imread(folder+"img_{:05}.png".format(i+1))
            ch = img[:,:,0]
            imgs.append(ch)
        return np.array(imgs)

    def load_labels(fn):
        return np.loadtxt(fn, usecols=0)

    pic = load_pics(folder+"/", n)
    ndata, width, height = pic.shape

    inp = (pic/np.float32(255)).reshape(n, width, height, 1)
    trg = load_labels(trgFile)
    trg = trg[0:n]

    return inp, trg, width, height

def loadDataAll(nTrn, nTst):
    # Load data
    (trnInp, trnTrg, imgW, imgH) = loadImages("polyAll-trn", "polyAll-trn_trg.csv", nTrn)
    (tstInp, tstTrg, imgW, imgH) = loadImages("polyAll-tst", "polyAll-tst_trg.csv", nTst)

    if K.image_data_format() == 'channels_first':
        trnInp = trnInp.reshape(trnInp.shape[0], 1, imgH, imgW)
        tstInp = tstInp.reshape(tstInp.shape[0], 1, imgH, imgW)
        input_shape = (1, imgH, imgW)
    else:
        trnInp = trnInp.reshape(trnInp.shape[0], imgH, imgW, 1)
        tstInp = tstInp.reshape(tstInp.shape[0], imgH, imgW, 1)
        input_shape = (imgH, imgW, 1)

    print('trnInp shape:', trnInp.shape)
    print('tstInp shape:', tstInp.shape)

    return trnInp, trnTrg, tstInp, tstTrg, input_shape

def loadDataOddEven(nTrn, nTst):
    # Load data
    (trnInp, trnTrg, imgW, imgH) = loadImages("polyOdd-trn", "polyOdd-trn_trg.csv", nTrn)
    (tstInp, tstTrg, imgW, imgH) = loadImages("polyEven-tst", "polyEven-tst_trg.csv", nTst)

    if K.image_data_format() == 'channels_first':
        trnInp = trnInp.reshape(trnInp.shape[0], 1, imgH, imgW)
        tstInp = tstInp.reshape(tstInp.shape[0], 1, imgH, imgW)
        input_shape = (1, imgH, imgW)
    else:
        trnInp = trnInp.reshape(trnInp.shape[0], imgH, imgW, 1)
        tstInp = tstInp.reshape(tstInp.shape[0], imgH, imgW, 1)
        input_shape = (imgH, imgW, 1)

    print('trnInp shape:', trnInp.shape)
    print('tstInp shape:', tstInp.shape)

    return trnInp, trnTrg, tstInp, tstTrg, input_shape


### CellName: PlotImg (#3)
### CellType: Information
### Cell instruction: Show some of the images

Here we look at the first ten pictures in the training set, and their respective targets. 

Run the cell by entering into the cell and press "CTRL Enter".

In [ ]:
trnInp, trnTrg, tstInp, tstTrg, input_shape = loadDataAll(10,10)
#trnInp, trnTrg, tstInp, tstTrg, input_shape = loadDataOddEven(10,10)

plt.figure(1, figsize=(15,10))
plt.imshow(trnInp[:10,:,:].swapaxes(0,1).reshape(input_shape[1],10*input_shape[1]),cmap="gray")
plt.axis("off")
plt.show()

print("Targets:")
print(trnTrg[:10])


### CellName: Stats (#4)
### CellType: Needed
### Cell instruction: Plot a confusion matrix

Run the cell by entering into the cell and press "CTRL Enter".

In [ ]:
def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()


### CellName: Ex1 (#5)
### CellType: Exercise
### Cell instruction: Instructions for question 1-2

## CNN for counting corners

#### Question 1
The question is simple! **Find a CNN model that can accurately counts the corners in the test data set.**
You have access to 5000 training images, but you may have to reduce that to be able to run it on your computer. In the
example code below I use 1000 training images.

**Hint 1** The cell below contains my simple model that is not so accurate. You can use the code as a starting point.
**Hint 2** How can we count corners? My suggestion is to solve this using a regression approach. The CNN will try
to predict the number of corners. So my output is a single linear node and I use MSE as a loss function. You can of
course also try to solve it using a classification approach, but then you will a problem, extrapolating to other
number of corners.
**Hint 3** The target data for the images are the number of corners. I rescale this by 10 before training.
**Hint 4** Cells #6 and #7 contains some code for visualizing the results of your model.

#### Question 2
There is a function "loadDataOddEven" that loads a training dataset with only odd number of corners and a test
dataset with only even number of corners. **Try to "solve" this problem with a CNN!**



In [ ]:
batch_size = 100
num_output = 1
epochs = 10

# Get the training data
(trnInp, trnTrg, tstInp, tstTrg, input_shape) = loadDataAll(1000,5000)
trnTrg /= 10;
tstTrg /= 10;

model = Sequential()

# Filter layer
model.add(Conv2D(10, (5,5), strides=2, padding='same', activation='relu', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))   

# Filter layer
model.add(Conv2D(10, (3,3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))   

# The dense layers
model.add(Flatten())
model.add(Dense(10, activation='relu'))
#model.add(Dropout(0.5))

model.add(Dense(num_output, activation='linear'))
    
model.compile(loss='mean_squared_error',
              optimizer=Adam(lr=0.001),
              metrics=['mse'])
model.summary()

estimator = model.fit(trnInp, trnTrg,
            batch_size=batch_size,
            epochs=epochs,
            verbose=1,
            validation_data=(tstInp, tstTrg))

trnSc = model.evaluate(trnInp, trnTrg, verbose=0)
tstSc = model.evaluate(tstInp, tstTrg, verbose=0)
print('Trn loss:', trnSc[0])
print('Tst loss:', tstSc[0])

# Plot the learning curves
plt.figure()
plt.plot(estimator.history['loss'], label='Training')
plt.plot(estimator.history['val_loss'], label='Validation')
plt.ylabel('MSE')
plt.xlabel('epoch')
plt.legend()
plt.show()


### CellName: Scatterplot (#6)
### CellType: Exercise
### Cell instruction: Make a scatter plot

This cell just makes two scatter plots between the predicted output and the target output, for both
training and test data.

In [ ]:
# Scatter plots of predicted and true values
trnPred = model.predict(trnInp)*10
tstPred = model.predict(tstInp)*10
trnTr = trnTrg*10
tstTr = tstTrg*10

plt.figure()
plt.plot(trnPred[:], trnTr[:], 'g.', label='Predict vs True (Training)')
plt.legend()

plt.figure()
plt.plot(tstPred[:], tstTr[:], 'g.', label='Predict vs True (Test)')
plt.legend()

plt.show()


### CellName: Confusion (#7)
### CellType: Exercise
### Cell instruction: Plot confusion matrix

This cell just plots the confusion matrices for both training and testdata. In order to make a "prediction" of the number of corners, the output from the CNN model is rounded to nearest integer.

In [ ]:
from sklearn.metrics import confusion_matrix

# Scatter plots of predicted and true values
trnPred = model.predict(trnInp)*10
tstPred = model.predict(tstInp)*10
trnTr = trnTrg*10
tstTr = tstTrg*10

nTrn = trnInp.shape[0]
nTst = tstInp.shape[0]

trnClPred = np.rint(trnPred).reshape(nTrn)
trnClPred = trnClPred.astype(int)
trnClTrg = np.rint(trnTr).astype(int)

tstClPred = np.rint(tstPred).reshape(nTst)
tstClPred = tstClPred.astype(int)
tstClTrg = np.rint(tstTr).astype(int)

minTrn = min(min(trnClTrg), min(trnClPred))
maxTrn = max(max(trnClTrg), max(trnClPred))
trgName = [];
for x in range(minTrn, maxTrn+1):
    trgName.append(str(x))

minTst = min(min(tstClTrg), min(tstClPred))
maxTst = max(max(tstClTrg), max(tstClPred))
tstName = [];
for x in range(minTst, maxTst+1):
    tstName.append(str(x))
    
confuTrn = confusion_matrix(trnClTrg, trnClPred)
confuTst = confusion_matrix(tstClTrg, tstClPred)

plot_confusion_matrix(cm           = confuTrn, 
                      normalize    = False,
                      target_names = trgName,
                      title        = "Confusion Matrix: Training")

plot_confusion_matrix(cm           = confuTst, 
                      normalize    = False,
                      target_names = tstName,
                      title        = "Confusion Matrix: Test")
